In [ ]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cols = ['Police Force Area', 'Year', 'Sex', 'Age group', 'Offence group', 'Sentence Outcome', 'Custodial Sentence Length','Sentenced']
df = pd.read_csv("/Users/alex/Downloads/obo_sent_pivot_2016_2022/obo_sent_pivot_2019_2022.csv", encoding= 'unicode_escape', low_memory=True, usecols=cols)
df.head()

In [ ]:
utils.lcColumns(df)
utils.renameColumns(df, columns={
    'year_of_appearance': 'year',
    'offence_group': 'offence',
    'police_force_area': 'pfa',
    'sentence_outcome': 'outcome',
    'custodial_sentence_length': 'sentence_length',
    'sentenced': 'freq',
    'count': 'freq'}
    )

In [ ]:
df.head()

In [ ]:
utils.tidy_elements(df)

In [ ]:
df.head()

In [ ]:
# Checking that sentence_length values have been tidied up
df['sentence_length'].unique()

In [ ]:
# Reordering columns
column_order = ['year', 'pfa', 'sex', 'age_group', 'offence', 'outcome', 'sentence_length', 'freq']
df = df.reindex(columns=column_order)

# Setting categorical columns
convert_dict = {'outcome': "category",
                'sentence_length': "category"
                }
df = df.astype(convert_dict)

In [ ]:
df.info()

In [ ]:
## FILTERING DATASET
filt1 = df['sex'] == 'Female'
filt2 = df['outcome'].isin(['Immediate custody', 'Community sentence','Suspended sentence'])
filt3 = df['age_group'].isin(["Adults", "Young adults"])
filt4 = df['pfa'].isin(["Special/miscellaneous and unknown police forces", "City of London", "Not known"])
filt = filt1 & filt2 & filt3 & ~filt4
women_dataset = df[filt].sort_values(['year', 'pfa']).copy()

In [ ]:
women_dataset.reset_index(drop=True, inplace=True)
women_dataset.head()

In [ ]:
len(women_dataset['pfa'].unique())

In [ ]:
women_dataset_2019 = women_dataset.query('year==2019')

Importing previous dataset to ensure matching

In [ ]:
df_orig = pd.read_csv('data/interim/PFA_2009-21_women_cust_comm_sus.csv')
df_orig.head()

In [ ]:
df_orig.query('year==2019').describe(include=['object', 'category'])

In [ ]:
df_orig_2019 = df_orig.query('year==2019')

In [ ]:
for column in women_dataset_2019.columns:
    print(column, women_dataset_2019[column].unique())

The only `outcome` appears to be `Community sentence` so something has gone wrong with the filtering stage. Let's circle back

In [ ]:
women_dataset_2019.info()

In [ ]:
women_dataset_2019.describe(include=['object', 'category'])

Right, that seems to confirm it. Time to examine the filtering again.

In [ ]:
df['outcome'].unique()

As ever, it's silly string formatting that seems to have tripped up everything. Let's rectify by making this case agnostic.

In [ ]:
mylist=['Immediate custody', 'Community sentence','Suspended sentence']
myset = set([s.lower() for s in mylist])
myset

In [ ]:
filt2 = df['outcome'].apply(lambda v: v.lower() in myset)
df[filt2].describe(include=['object', 'category'])

Re-filtering dataframe

In [ ]:
## FILTERING DATASET
sentences = ['Immediate custody', 'Community sentence','Suspended sentence']
sentences_set = set([s.lower() for s in sentences])

filt1 = df['sex'] == 'Female'
filt2 = df['outcome'].apply(lambda v: v.lower() in sentences_set)
filt3 = df['age_group'].isin(["Adults", "Young adults"])
filt4 = df['pfa'].isin(["Special/miscellaneous and unknown police forces", "City of London", "Not known"])
filt = filt1 & filt2 & filt3 & ~filt4
women_dataset = df[filt].sort_values(['year', 'pfa']).copy()

In [ ]:
women_dataset.describe(include=['object', 'category'])

Great, we have three unique values in the `outcome` column. Let's filter for 2019 and check further

In [ ]:
women_dataset_2019 = women_dataset.query('year==2019')

In [ ]:
women_dataset_2019.info()

In [ ]:
women_dataset_2019['freq'].sum()

In [ ]:
df_orig_2019['freq'].sum()

Right, the sum of `freq` is the same for both dataframes. Let's now check against all the years within `women_dataset` and `df_orig`

In [ ]:
df_orig['year'].unique()

In [ ]:
women_dataset['year'].unique()

In [ ]:
for year in women_dataset['year'].unique():
    print(year, women_dataset.query(f'year=={year}')['freq'].sum())

In [ ]:
for year in women_dataset['year'].unique():
    sentenced_sum = df_orig.query(f'year=={year}')['freq'].sum()
    if sentenced_sum > 0:
        print(year, sentenced_sum)

## Excellent, think we're there. Time to bring this all together.

In [ ]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np

In [ ]:
cols = ['Police Force Area', 'Year', 'Sex', 'Age group', 'Offence group', 'Sentence Outcome', 'Custodial Sentence Length','Sentenced']
df = pd.read_csv("/Users/alex/Downloads/obo_sent_pivot_2016_2022/obo_sent_pivot_2019_2022.csv", encoding= 'unicode_escape', low_memory=True, usecols=cols)

In [ ]:
utils.lcColumns(df)
utils.renameColumns(df, columns={
    'year_of_appearance': 'year',
    'offence_group': 'offence',
    'police_force_area': 'pfa',
    'sentence_outcome': 'outcome',
    'custodial_sentence_length': 'sentence_length',
    'sentenced': 'freq',
    'count': 'freq'}
    )
utils.tidy_elements(df)

# Reordering columns
column_order = ['year', 'pfa', 'sex', 'age_group', 'offence', 'outcome', 'sentence_length', 'freq']
df = df.reindex(columns=column_order)

# Setting categorical columns
convert_dict = {'outcome': "category",
                'sentence_length': "category"
                }
df = df.astype(convert_dict)

In [ ]:
## FILTERING DATASET
sentences = ['Immediate custody', 'Community sentence','Suspended sentence']
sentences_set = set([s.lower() for s in sentences])

filt1 = df['sex'] == 'Female'
filt2 = df['outcome'].apply(lambda v: v.lower() in sentences_set)
filt3 = df['age_group'].isin(["Adults", "Young adults"])
filt4 = df['pfa'].isin(["Special/miscellaneous and unknown police forces", "City of London", "Not known"])
filt = filt1 & filt2 & filt3 & ~filt4
women_dataset = df[filt].sort_values(['year', 'pfa']).copy()

In [ ]:
women_dataset.head()

In [ ]:
women_dataset['year'].unique()

In [ ]:
women_dataset.query('year==2019')['freq'].sum()

Right, that looks to have behaved. Let's now attempt to run the full `data_cleansing.py` code below, with those filtering adaptations made.

In [1]:
import src.data.utilities as utils
%load_ext autoreload
%autoreload 2

import pandas as pd
import glob

Building a simple list to read in multiple csv files

In [2]:
path="data/external/obo_sent_pivot_2016_2022/"
all_files = glob.glob(path + "*.csv")
all_files

['data/external/obo_sent_pivot_2016_2022/obo_sent_pivot_2019_2022.csv',
 'data/external/obo_sent_pivot_2016_2022/obo_sent_pivot_2016_2018.csv']

In [3]:
cols = ['Police Force Area', 'Year', 'Sex', 'Age group', 'Offence group', 'Sentence Outcome', 'Custodial Sentence Length','Sentenced']
all_csvs = [utils.loadData(filename, cols=cols) for filename in all_files]
df = pd.concat(all_csvs, axis=0, ignore_index=True)

Checking that the concat has pulled through all of the dates

In [4]:
df['Year'].unique()

array([2019, 2020, 2021, 2022, 2016, 2017, 2018])

Importing other csv and working through the `data_cleansing.py` stages

In [5]:
# 2. Court outcomes by police force area 2009–2019 (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/888561/csvs-behind-data-tools-2-2019.zip)
cols_2009 =['Police Force Area', 'Year of Appearance', 'Sex', 'Age Group', 'Offence Group', 'Outcome', 'Custodial Sentence Length','Count'] 
df_2009 = utils.loadData('../womens-pfa-analysis/data/external/court-outcomes-by-PFA-2019.csv', cols=cols_2009)

In [6]:
df_2009['Year of Appearance'].unique()

array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019])

In [7]:
filt = df_2009['Year of Appearance'] < 2016
df_2009 = df_2009[filt].copy()
df_2009['Year of Appearance'].unique()

array([2009, 2010, 2011, 2012, 2013, 2014, 2015])

In [8]:
df_list = utils.dataframeList(locals())

In [9]:
# Renaming columns
for data in df_list:
    utils.lcColumns(data)
    utils.renameColumns(data, columns={
        'year_of_appearance': 'year',
        'offence_group': 'offence',
        'police_force_area': 'pfa',
        'sentence_outcome': 'outcome',
        'custodial_sentence_length': 'sentence_length',
        'sentenced': 'freq',
        'count': 'freq'}
        )

# Joining and tidying elements into one DataFrame
df_combined = pd.concat(df_list)
utils.tidy_elements(df_combined)

# Reordering columns
column_order = ['year', 'pfa', 'sex', 'age_group', 'offence', 'outcome', 'sentence_length', 'freq']
df_combined = df_combined.reindex(columns=column_order)

In [10]:
df_combined

,year,pfa,sex,age_group,offence,outcome,sentence_length,freq
0,2019,West Yorkshire,Female,Children,Violence against the person,Community sentence,24:Not known,1
1,2019,Metropolitan Police,Female,Children,Robbery,Immediate Custody,More than 3 months and up to 6 months,2
2,2019,Gloucestershire,Female,Children,Violence against the person,Community sentence,24:Not known,1
3,2019,Derbyshire,Female,Children,Theft offences,Community sentence,24:Not known,1
4,2019,Greater Manchester,Female,Children,Miscellaneous crimes against society,Community sentence,24:Not known,1
...,...,...,...,...,...,...,...,...
4085619,2015,Wiltshire,"Companies, public bodies etc.","Companies, public bodies etc.",Summary motoring,Fine,NaN,1
4085620,2015,Wiltshire,"Companies, public bodies etc.","Companies, public bodies etc.",Summary motoring,Withdrawn,NaN,1
4085621,2015,Wiltshire,"Companies, public bodies etc.","Companies, public bodies etc.",Summary non-motoring,Fine,NaN,3
4085622,2015,Wiltshire,"Companies, public bodies etc.","Companies, public bodies etc.",Miscellaneous crimes against society,Fine,NaN,1


In [11]:
convert_dict = {'outcome': "category",
                'sentence_length': "category"
                }
df_combined = df_combined.astype(convert_dict)

Capitalising the outcomes for consistency before filtering

In [36]:
df_combined['outcome'] = df_combined['outcome'].str.capitalize()
df_combined['outcome'].unique()

array(['Community sentence', 'Immediate custody', 'Otherwise dealt with',
       'Conditional discharge', 'Absolute discharge', 'Fine',
       'Compensation (primary disposal)', 'Suspended sentence',
       'Committed for trial', 'Proceeding discontinued', 'Withdrawn',
       'Dismissed', 'Acquitted crown court', 'Discharged sec 6',
       'Compensation', 'Not tried crown court', 'Committed for sentence'],
      dtype=object)

In [37]:
df_combined['outcome'] = df_combined['outcome'].str.capitalize()

## FILTERING DATASET
filt1 = df_combined['sex'] == 'Female'
filt2 = df_combined['outcome'].isin(['Immediate custody', 'Community sentence','Suspended sentence'])
filt3 = df_combined['age_group'].isin(["Adults", "Young adults"])
filt4 = df_combined['pfa'].isin(["Special/miscellaneous and unknown police forces", "City of London", "Not known"])
filt = filt1 & filt2 & filt3 & ~filt4
women_dataset = df_combined[filt].sort_values(['year', 'pfa']).copy()

In [39]:
women_dataset['outcome'].unique()

array(['Community sentence', 'Suspended sentence', 'Immediate custody'],
      dtype=object)

In [41]:
women_dataset.query('year==2022')['freq'].sum()

19182

In [42]:
for year in women_dataset['year'].unique():
    sentenced_sum = women_dataset.query(f'year=={year}')['freq'].sum()
    if sentenced_sum > 0:
        print(year, sentenced_sum)

2009 36925
2010 37966
2011 37783
2012 33929
2013 31282
2014 31124
2015 32163
2016 29795
2017 28206
2018 25698
2019 24352
2020 17377
2021 20275
2022 19182


Running this again, but using the 2010–2015 data also provided in the same publication, rather than the older 2009–2019 data. I've also renamed the folder to reference that the data goes back to 2010 now—so earlier code will throw and error.

This means I can lose quite a bit of the stages (commented out below)

In [43]:
## Collecting file paths with glob

path="data/external/obo_sent_pivot_2010_2022/"
all_files = glob.glob(path + "*.csv")


## IMPORTING DATASETS ##
# 1. Sentencing data 2010–22 
# (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1157979/obo_sent_pivot_2010_2015.zip and 
# https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1157991/obo_sent_pivot_2016_2022.zip)

cols = ['Police Force Area', 'Year', 'Sex', 'Age group', 'Offence group', 'Sentence Outcome', 'Custodial Sentence Length','Sentenced']
all_csvs = [utils.loadData(filename, cols=cols) for filename in all_files]
df = pd.concat(all_csvs, axis=0, ignore_index=True)

# # 2. Court outcomes by police force area 2009–2019 (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/888561/csvs-behind-data-tools-2-2019.zip)
# cols_2009 =['Police Force Area', 'Year of Appearance', 'Sex', 'Age Group', 'Offence Group', 'Outcome', 'Custodial Sentence Length','Count'] 
# df_2009 = utils.loadData('../womens-pfa-analysis/data/external/court-outcomes-by-PFA-2019.csv', cols=cols_2009)

# #Dropping duplicate data from 2009 dataset that also appears in df
# filt = df_2009['Year of Appearance'] < 2016
# df_2009 = df_2009[filt].copy()

# Defining datasets to iterate through in following section
# df_list = utils.dataframeList(locals()) #Using locals() function to retrieve local symbol table. Note this outputs a complex list, and is no longer a DataFrame.

## DATA CLEANING PROCESS

# Renaming columns
utils.lcColumns(df)
utils.renameColumns(df, columns={
    'year_of_appearance': 'year',
    'offence_group': 'offence',
    'police_force_area': 'pfa',
    'sentence_outcome': 'outcome',
    'custodial_sentence_length': 'sentence_length',
    'sentenced': 'freq',
    'count': 'freq'}
    )

# Tidying elements into one DataFrame
utils.tidy_elements(df)

# Reordering columns
column_order = ['year', 'pfa', 'sex', 'age_group', 'offence', 'outcome', 'sentence_length', 'freq']
df = df.reindex(columns=column_order)

# Setting categorical columns
convert_dict = {'outcome': "category",
                'sentence_length': "category"
                }
df = df.astype(convert_dict)

# Setting outcomes to lowercase
df['outcome'] = df['outcome'].str.capitalize()

## FILTERING DATASET
filt1 = df['sex'] == 'Female'
filt2 = df['outcome'].isin(['Immediate custody', 'Community sentence','Suspended sentence'])
filt3 = df['age_group'].isin(["Adults", "Young adults"])
filt4 = df['pfa'].isin(["Special/miscellaneous and unknown police forces", "City of London", "Not known"])
filt = filt1 & filt2 & filt3 & ~filt4
women_dataset = df[filt].sort_values(['year', 'pfa']).copy()

In [44]:
for year in women_dataset['year'].unique():
    sentenced_sum = women_dataset.query(f'year=={year}')['freq'].sum()
    if sentenced_sum > 0:
        print(year, sentenced_sum)

2010 37966
2011 37783
2012 33929
2013 31282
2014 31124
2015 32163
2016 29795
2017 28206
2018 25698
2019 24352
2020 17377
2021 20275
2022 19182


In [45]:
women_dataset.to_csv('data/interim/TESTING_PFA_2010-22_women_cust_comm_sus.csv', index=False)

### HERE
Think I will add 2009 data back in from the other file next, just for continuity.

In [40]:
## Collecting file paths with glob

path="data/external/obo_sent_pivot_2010_2022/"
all_files = glob.glob(path + "*.csv")


## IMPORTING DATASETS ##
# 1. Sentencing data 2010–22 
# (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1157979/obo_sent_pivot_2010_2015.zip and 
# https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1157991/obo_sent_pivot_2016_2022.zip)

cols = ['Police Force Area', 'Year', 'Sex', 'Age group', 'Offence group', 'Sentence Outcome', 'Custodial Sentence Length','Sentenced']
all_csvs = [utils.loadData(filename, cols=cols) for filename in all_files]
df = pd.concat(all_csvs, axis=0, ignore_index=True)

# # 2. Court outcomes by police force area 2009–2019 (from: https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/888561/csvs-behind-data-tools-2-2019.zip)
# cols_2009 =['Police Force Area', 'Year of Appearance', 'Sex', 'Age Group', 'Offence Group', 'Outcome', 'Custodial Sentence Length','Count'] 
# df_2009 = utils.loadData('../womens-pfa-analysis/data/external/court-outcomes-by-PFA-2019.csv', cols=cols_2009)

# #Dropping duplicate data from 2009 dataset that also appears in df
# filt = df_2009['Year of Appearance'] < 2016
# df_2009 = df_2009[filt].copy()

# Defining datasets to iterate through in following section
# df_list = utils.dataframeList(locals()) #Using locals() function to retrieve local symbol table. Note this outputs a complex list, and is no longer a DataFrame.

## DATA CLEANING PROCESS

# Renaming columns
utils.lcColumns(df)
utils.renameColumns(df, columns={
    'year_of_appearance': 'year',
    'offence_group': 'offence',
    'police_force_area': 'pfa',
    'sentence_outcome': 'outcome',
    'custodial_sentence_length': 'sentence_length',
    'sentenced': 'freq',
    'count': 'freq'}
    )

# Tidying elements into one DataFrame
utils.tidy_elements(df)

# Reordering columns
column_order = ['year', 'pfa', 'sex', 'age_group', 'offence', 'outcome', 'sentence_length', 'freq']
df = df.reindex(columns=column_order)

# Setting categorical columns
convert_dict = {'outcome': "category",
                'sentence_length': "category"
                }
df = df.astype(convert_dict)

# Setting outcomes to lowercase
df['outcome'] = df['outcome'].str.capitalize()

## FILTERING DATASET
filt1 = df['sex'] == 'Female'
filt2 = df['outcome'].isin(['Immediate custody', 'Community sentence','Suspended sentence'])
filt3 = df['age_group'].isin(["Adults", "Young adults"])
filt4 = df['pfa'].isin(["Special/miscellaneous and unknown police forces", "City of London", "Not known"])
filt = filt1 & filt2 & filt3 & ~filt4
women_dataset = df[filt].sort_values(['year', 'pfa']).copy()

## OUTPUTTING INTERIM DATASET FOR FURTHER ANALYSIS
women_dataset.to_csv('data/interim/TESTING_PFA_2010-22_women_cust_comm_sus.csv', index=False)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.